In [1]:
pip install transformers datasets pandarallel beautifulsoup4

     |████████████████████████████████| 4.0 MB 8.5 MB/s 
     |████████████████████████████████| 342 kB 71.0 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 880 kB 33.9 MB/s 
     |████████████████████████████████| 6.6 MB 41.4 MB/s 
     |████████████████████████████████| 596 kB 48.2 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 136 kB 63.5 MB/s 
     |████████████████████████████████| 212 kB 66.0 MB/s 
     |████████████████████████████████| 127 kB 25.4 MB/s 
     |████████████████████████████████| 271 kB 64.8 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=e88893052be0f0acba735eb3c160c96688c38e579502df03a4c0e12b4a1c86dc
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f

## Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd

import torch
from transformers import GPT2LMHeadModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


0it [00:00, ?it/s]

cuda


In [3]:
torch.cuda.empty_cache()

## Import GPT Model

In [4]:
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Sberbank GPT model doesn't have eos_token, so we can add it manually:


In [5]:
if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '<|endoftext|>'})

model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 1024)

In [6]:
# checking eos_token token
tokenizer.eos_token

'<|endoftext|>'

## Import Dataset to Pandas

In [8]:
# import csv
df = pd.read_csv('/content/toasts.csv', delimiter='\t', header = None)
df = df.rename({0: 'Toast'}, axis='columns')
df.head(5)

,Toast
0,Будь здоров сто веков!
1,Самый короткий тост: Enter!
2,За свою семью рюмочку я пью!
3,"Лучше поздно, чем не до дна!"
4,"Кроме здоровья, денег и интима!"


In [9]:
# the number of tokens in a row must not exceed 1024 (350)
# let's check the maximum value
df['token_count'] = df["Toast"].str.replace(',','').str.split().str.len()
assert df['token_count'].max() < 350
print(df['token_count'].max())

266


In [10]:
# delete the token_count column
df = df.drop('token_count', 1)

## Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 42)

# reset indexes
train_df = train_df.reset_index()
train_df = train_df.drop('index', 1)

test_df = test_df.reset_index()
test_df = test_df.drop('index', 1)

In [12]:
print(train_df.shape)
print(test_df.shape)

(858, 1)
(215, 1)


In [13]:
#For the test set only, keep last 6 words in a new column, then remove them from original column
test_df['ToastEnd'] = test_df['Toast'].str.split().str[-5:].apply(' '.join)
test_df['ToastHead'] = test_df['Toast'].str.split().str[:-5].apply(' '.join)

In [14]:
test_df

,Toast,ToastEnd,ToastHead
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман..."
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...
...,...,...,...
210,Муж наливает себе рюмку водки. Жена говорит: –...,– никогда! За Новый год!,Муж наливает себе рюмку водки. Жена говорит: –...
211,"Все мы учились, все мы должны были поумнеть. К...",году всегда делать правильный выбор!,"Все мы учились, все мы должны были поумнеть. К..."
212,"Говорят, что с кем Новый год встретишь с тем е...",все сказано. С Новым годом.,"Говорят, что с кем Новый год встретишь с тем е..."
213,Николай Васильевич Гоголь писал: «Отец любит с...,же за родство наших душ!,Николай Васильевич Гоголь писал: «Отец любит с...


## Create the Dataset for GPT

In [15]:
class ToastsDataset(Dataset):
    def __init__(self, toast_df, eos_token = "<|endoftext|>", column = "Toast"):
        super().__init__()

        self.toast_df = toast_df
        self.eos_token = eos_token
        self.column = column
                
    def __len__(self):
        return len(self.toast_df)

    def __getitem__(self, idx):
        row = self.toast_df[self.column].iat[idx]

        return f"ТОСТ:{row}{self.eos_token}"

In [16]:
train_dataset = ToastsDataset(toast_df = train_df)

## Generation Checking before Training

A bleu score can be used to check the generation on a test dataset. At this point I want to check that the score actually works and evaluate how poor it is.

**Note**: maybe it's better to use in combination with *nist-score* and *meteor_score*.

In [17]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count = 10,
    entry_length = 64, #maximum number of words
    top_p = 0.8,
    temperature = 1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in range(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().cpu().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().cpu().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [18]:
test_df['Generated'] = test_df['ToastHead'].progress_apply(lambda x: generate(model, tokenizer, prompt = x, entry_count = 1)[0])

  0%|          | 0/215 [00:00<?, ?it/s]

In [19]:
test_df

,Toast,ToastEnd,ToastHead,Generated
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман..."
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...
...,...,...,...,...
210,Муж наливает себе рюмку водки. Жена говорит: –...,– никогда! За Новый год!,Муж наливает себе рюмку водки. Жена говорит: –...,Муж наливает себе рюмку водки. Жена говорит: –...
211,"Все мы учились, все мы должны были поумнеть. К...",году всегда делать правильный выбор!,"Все мы учились, все мы должны были поумнеть. К...","Все мы учились, все мы должны были поумнеть. К..."
212,"Говорят, что с кем Новый год встретишь с тем е...",все сказано. С Новым годом.,"Говорят, что с кем Новый год встретишь с тем е...","Говорят, что с кем Новый год встретишь с тем е..."
213,Николай Васильевич Гоголь писал: «Отец любит с...,же за родство наших душ!,Николай Васильевич Гоголь писал: «Отец любит с...,Николай Васильевич Гоголь писал: «Отец любит с...


In [20]:
from bs4 import BeautifulSoup

test_df['Generated'] = test_df['Generated'].apply(lambda x: BeautifulSoup(x).getText())
test_df.head(5)

,Toast,ToastEnd,ToastHead,Generated
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман..."
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...


In [21]:
#remove \n\n
test_df['Generated'] = test_df['Generated'].str.replace("\n", " ")
#remove \n\n
test_df['Generated'] = test_df['Generated'].str.replace("\t", " ")
#remove double spaces
test_df['Generated'] = test_df['Generated'].str.replace(" +", " ")
test_df.head(5)

,Toast,ToastEnd,ToastHead,Generated
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман..."
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...


In [22]:
test_df.head(5)

,Toast,ToastEnd,ToastHead,Generated
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман..."
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...


In [23]:
import statistics
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.translate.bleu_score import SmoothingFunction

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

In [26]:
test_df["GeneratedEnd"] = test_df.progress_apply(lambda row: remove_prefix(row["Generated"], row["ToastHead"]), axis=1)

  0%|          | 0/215 [00:00<?, ?it/s]

In [27]:
smoothie = SmoothingFunction().method2

def calc_bleu(reference, candidate):
  reference = word_tokenize(reference, language="russian")
  candidate = word_tokenize(candidate, language="russian")

  return sentence_bleu([reference], candidate, smoothing_function=smoothie)

In [28]:
test_df['BLEU'] = test_df.progress_apply(lambda row: calc_bleu(row['ToastEnd'],row['GeneratedEnd']), axis=1)
test_df.sample(5)

  0%|          | 0/215 [00:00<?, ?it/s]

,Toast,ToastEnd,ToastHead,Generated,GeneratedEnd,BLEU
214,Грета читает антологию любовной лирики. Неожид...,заставляет всех мужчин мыслить идентично!,Грета читает антологию любовной лирики. Неожид...,Грета читает антологию любовной лирики. Неожид...,Грета читает антологию любовной лирики. Неожид...,0.014243
193,"Новый год – самый яркий, самый красивый, торже...","щедрой, как праздник Нового года!","Новый год – самый яркий, самый красивый, торже...","Новый год – самый яркий, самый красивый, торже...","Новый год – самый яркий, самый красивый, торже...",0.011804
17,"Многие, если выпьют, то и получку до дома доне...",выпить за таких хозяйственных мужиков!,"Многие, если выпьют, то и получку до дома доне...","Многие, если выпьют, то и получку до дома доне...","Многие, если выпьют, то и получку до дома доне...",0.019905
38,"Извини, опоздал! Надеюсь, за то время, что мен...","ты не опаздывал, как я!","Извини, опоздал! Надеюсь, за то время, что мен...","Извини, опоздал! Надеюсь, за то время, что мен...","Извини, опоздал! Надеюсь, за то время, что мен...",0.023414
44,"Мужчина молод столько, сколько он любит. Женщи...",выпьем же за нашу молодость!,"Мужчина молод столько, сколько он любит. Женщи...","Мужчина молод столько, сколько он любит. Женщи...","Мужчина молод столько, сколько он любит. Женщи...",0.000000


In [29]:
test_df.BLEU.mean()

0.016425828361410866

## Hyperparameters

In [30]:
BATCH_SIZE = 16
EPOCHS = 31
LEARNING_RATE = 2e-5
WARMUP_STEPS = 200
MAX_SEQ_LEN = 200

In [31]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1)

## Model training

In [32]:
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)
proc_seq_count = 0
sum_loss = 0.0
min_sum_loss = 99999999
batch_count = 0

tmp_toasts_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    
    print(f"EPOCH: {epoch} Started " + '=' * 5)
    
    for idx, toasts in enumerate(train_loader):
        
        #################### "Fit as many joke sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        toast_tens = torch.tensor(tokenizer.encode(toasts[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if toast_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        #The first joke sequence in the sequence
        if not torch.is_tensor(tmp_toasts_tens):
            tmp_toasts_tens = toast_tens
            continue
        else:
            #The next joke does not fit in so we process the sequence and leave the last joke 
            #as the start for next sequence 
            if tmp_toasts_tens.size()[1] + toast_tens.size()[1] > MAX_SEQ_LEN:
                work_toasts_tens = tmp_toasts_tens
                tmp_toasts_tens = toast_tens
            else:
                #Add the joke to sequence, continue and try to add more
                tmp_toasts_tens = torch.cat([tmp_toasts_tens, toast_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################
        outputs = model(work_toasts_tens, labels=work_toasts_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss += loss.detach().data
                       
        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

    print(f"Sum loss {sum_loss}")
    sum_loss = 0.0
    # save every fifth epoch
    if epoch % 5 == 0:
      torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_toast_{epoch}.pt"))

torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_toast_{epoch}.pt"))

EPOCH: 0 Started =====
Sum loss 1506.2315673828125
EPOCH: 1 Started =====
Sum loss 1278.9989013671875
EPOCH: 2 Started =====
Sum loss 1158.8060302734375
EPOCH: 3 Started =====
Sum loss 1079.63232421875
EPOCH: 4 Started =====
Sum loss 1037.2530517578125
EPOCH: 5 Started =====
Sum loss 996.2490844726562
EPOCH: 6 Started =====
Sum loss 930.9153442382812
EPOCH: 7 Started =====
Sum loss 903.92333984375
EPOCH: 8 Started =====
Sum loss 864.8489990234375
EPOCH: 9 Started =====
Sum loss 828.7530517578125
EPOCH: 10 Started =====
Sum loss 832.3624877929688
EPOCH: 11 Started =====
Sum loss 827.5980224609375
EPOCH: 12 Started =====
Sum loss 841.3050537109375
EPOCH: 13 Started =====
Sum loss 841.118408203125
EPOCH: 14 Started =====
Sum loss 839.9132690429688
EPOCH: 15 Started =====
Sum loss 830.9718017578125
EPOCH: 16 Started =====
Sum loss 836.7991333007812
EPOCH: 17 Started =====
Sum loss 848.2129516601562
EPOCH: 18 Started =====
Sum loss 839.8168334960938
EPOCH: 19 Started =====
Sum loss 841.2924

In [33]:
torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_toast_{epoch}.pt"))

## Generation After Training 

Copying here my previous code from the above to check the trained model.

**Note:** come up with a better solution, rather than repeating code that has already been written

In [34]:
test_df['Generated'] = test_df['ToastHead'].progress_apply(lambda x: generate(model, tokenizer, prompt = x, entry_count = 1)[0])

  0%|          | 0/215 [00:00<?, ?it/s]

In [35]:
# beautiful soup

test_df['Generated'] = test_df['Generated'].apply(lambda x: BeautifulSoup(x).getText())
test_df.head(5)

,Toast,ToastEnd,ToastHead,Generated,GeneratedEnd,BLEU
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...,0.008200
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...,0.009952
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...,0.004885
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман...",0.015576
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...,0.015457


In [36]:
#remove \n\n
test_df['Generated'] = test_df['Generated'].str.replace("\n", " ")
#remove \n\n
test_df['Generated'] = test_df['Generated'].str.replace("\t", " ")
#remove double spaces
test_df['Generated'] = test_df['Generated'].str.replace(" +", " ")
test_df.head(5)

,Toast,ToastEnd,ToastHead,Generated,GeneratedEnd,BLEU
0,Существуют две системы выпить: первая – выпить...,пью за обаяние милейшей хозяйки!,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...,Существуют две системы выпить: первая – выпить...,0.008200
1,– Дед Мороз! – кричит маленький мальчик за нес...,он становится для нас праздником!,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...,– Дед Мороз! – кричит маленький мальчик за нес...,0.009952
2,Два друга приехали на рыбалку. Разожгли костер...,необходимые документы были в порядке!,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...,Два друга приехали на рыбалку. Разожгли костер...,0.004885
3,"– Спасибо, доктор, что вы вылечили меня от ман...",в новом году быть здоровыми!,"– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман...","– Спасибо, доктор, что вы вылечили меня от ман...",0.015576
4,Испанцы любят желать своим друзьям и родственн...,мы нашему имениннику того же!,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...,Испанцы любят желать своим друзьям и родственн...,0.015457


In [38]:
test_df["GeneratedEnd"] = test_df.progress_apply(lambda row: remove_prefix(row["Generated"], row["ToastHead"]), axis=1)

  0%|          | 0/215 [00:00<?, ?it/s]

In [39]:
test_df['BLEU'] = test_df.progress_apply(lambda row: calc_bleu(row['ToastEnd'],row['GeneratedEnd']), axis=1)
test_df.sample(5)

  0%|          | 0/215 [00:00<?, ?it/s]

,Toast,ToastEnd,ToastHead,Generated,GeneratedEnd,BLEU
100,"Да здравствует всё то, благодаря чему мы все, ...","все, несмотря ни на что!","Да здравствует всё то, благодаря чему мы","Да здравствует всё то, благодаря чему мы имеем...",имеем сегодня прекрасную возможность полноцен...,0.040519
166,"Выпьем за здоровье нашего хозяина и его семьи,...",была беременность его любимой жены.,"Выпьем за здоровье нашего хозяина и его семьи,...","Выпьем за здоровье нашего хозяина и его семьи,...","не появилась в их доме. - Кстати, на днях он ...",0.025043
42,Древнегреческий публицист Сократ еще в пятом в...,отношения! Пусть они будут долгими!,Древнегреческий публицист Сократ еще в пятом в...,Древнегреческий публицист Сократ еще в пятом в...,отношения! виртуальная экскурсия по «старой д...,0.035416
132,"Охотник гонится за зайцем. – Дядя, а не видел ...",вовремя оказываться в нужном месте!,"Охотник гонится за зайцем. – Дядя, а не видел ...","Охотник гонится за зайцем. – Дядя, а не видел ...",мы чаще заглядывали в будущее!,0.244462
59,"Желаем, чтобы этот контраст был пронесен через...",всегда было солнечно и тепло!,"Желаем, чтобы этот контраст был пронесен через...","Желаем, чтобы этот контраст был пронесен через...",у нас всегда тепло и радостно. Пусть мы окруж...,0.025486


In [40]:
test_df.BLEU.mean()

0.07549579440244346